In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [8]:
!pip install openpyxl

In [ ]:
!pip install nltk

In [ ]:
!pip install "https://github.com/megagonlabs/ginza/releases/download/latest/ginza-latest.tar.gz"

In [ ]:
!pip install spacy==2.3.0

In [ ]:
!pip install -U ginza ja_ginza

In [2]:
import os
import datetime
import codecs
import re
import warnings
warnings.filterwarnings('ignore')
import pkg_resources, imp
imp.reload(pkg_resources)
import numpy as np
import pandas as pd
import holoviews as hv
from holoviews import opts
hv.extension('bokeh')
from matplotlib import pyplot as plt
# import neologdn
import nltk
from nltk.corpus import stopwords
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import spacy
from spacy.lang.ja import Japanese
import regex
from wordcloud import WordCloud
import collections
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import lightgbm as lgb
# import shap
# shap.initjs()
# import oseti

In [4]:
import os
os.getcwd()
print(os.getcwd())

In [5]:
import re
import itertools
import unicodedata
from janome.tokenizer import Tokenizer
import networkx as nx
from scipy.spatial import distance
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'Droid Sans Japanese'

In [7]:
import pandas as pd

# Load the dataset into a pandas dataframe.
# df = pd.read_csv("./cola_public/raw/in_domain_train.tsv", delimiter='\t', header=None, names=['sentence_source', 'label', 'label_notes', 'sentence'])
# list=[10,11,15,16]
col_list=[15]
df = pd.read_excel("/kaggle/input/ja-nlp/ja_slide9_data.xlsx", usecols=col_list,names=['sentence'])
# Report the number of sentences.
print('Number of training sentences: {:,}\n'.format(df.shape[0]))

# Display 10 random rows from the data.
df.sample(5)

In [ ]:
tweets = df['sentence'].as_matrix()

regex = u'[^\u3041-\u3093\u30A1-\u30F4\u4E00-\u9FCB]'
t = Tokenizer()

tweet_words = []
for tweet in tweets:
    tweet = re.sub(regex, ' ', tweet)
    words = []
    for token in t.tokenize(tweet):
        speechs = token.part_of_speech.split(',')
        if ('名詞' in speechs) or ('形容詞' in speechs):
            if len(token.surface) > 1:
                words.append(token.surface)
    tweet_words.append(words)

len(tweet_words)

In [ ]:
tweet_words

In [ ]:
word_cnt = {}
for words in tweet_words:
    for word in words:
        if word not in word_cnt:
            word_cnt[word] = 1
        else:
            word_cnt[word] += 1
    
word_cnt_df = pd.DataFrame({'word': [k for k in word_cnt.keys()], 'cnt': [v for v in word_cnt.values()]})
len(word_cnt_df)

In [ ]:
# word_cnt_df

In [ ]:
word_cnt_df[['cnt']].describe()

In [ ]:
tmp = word_cnt_df[word_cnt_df['cnt'] > 3]
tmp.sort_values(by='cnt', ascending=False).plot(kind='bar', x='word', y='cnt', figsize=(15,7), legend=False)
plt.show()

### 两个词的

In [ ]:
def ja_preprocess(text):
    nlp = spacy.load('ja_ginza')
    lowered = text.lower()
    normalized = neologdn.normalize(lowered)
    tokenized = []
    for i in nlp(normalized):
        word = i.lemma_
        pos = i.pos_
        if pos in ["NOUN", "ADJ", "VERB", "ADV", "PROPN"] and len(word) > 1:
#         and i.text not in stopwords_ja:
            tokenized.append(word)
    preprocessed = " ".join(tokenized)
    return preprocessed

In [ ]:
df_preprocessed['ja_preprocessed'] = df['sentence'].apply(lambda x: ja_preprocess(x))

In [ ]:
def ngram_func(ngram, text_series):
    string_filterd =  text_series.sum().split()
    dic = nltk.FreqDist(nltk.ngrams(string_filterd, ngram)).most_common(30)
    ngram_df = pd.DataFrame(dic, columns=['ngram','count'])
    ngram_df.index = [' '.join(i) for i in ngram_df.ngram]
    ngram_df.drop('ngram',axis=1, inplace=True)
    return ngram_df
uni_ja = hv.Bars(ngram_func(1, df_preprocessed['ja_preprocessed'])[::-1]).opts(title="Japanese Unigram Count top-30", color="red", xlabel="Unigrams", ylabel="Count")
bi_ja = hv.Bars(ngram_func(2, df_preprocessed['ja_preprocessed'])[::-1]).opts(title="Japanese Bigram Count top-30", color="blue", xlabel="Bigrams", ylabel="Count")
(uni_ja + bi_ja).opts(opts.Bars(width=380, height=600,tools=['hover'],show_grid=True,invert_axes=True,fontsize={'title':10})).opts(shared_axes=False)

In [ ]:
# tweet_combinations

In [ ]:
vocab = {}
target_words = word_cnt_df[word_cnt_df['cnt'] > 3]['word'].as_matrix()
for word in target_words:
    if word not in vocab:
        vocab[word] = len(vocab)

re_vocab = {}
for word, i in vocab.items():
    re_vocab[i] = word
    
len(vocab)

In [ ]:
tweet_combinations = [list(itertools.combinations(words, 2)) for words in tweet_words]
combination_matrix = np.zeros((len(vocab), len(vocab)))

for tweet_comb in tweet_combinations:
    for comb in tweet_comb:
        if comb[0] in target_words and comb[1] in target_words:
            combination_matrix[vocab[comb[0]], vocab[comb[1]]] += 1
            combination_matrix[vocab[comb[1]], vocab[comb[0]]] += 1
        
for i in range(len(vocab)):
    combination_matrix[i, i] /= 2
        
combination_matrix

In [ ]:
jaccard_matrix = 1 - distance.cdist(combination_matrix, combination_matrix, 'jaccard')
jaccard_matrix

In [ ]:
nodes = []

for i in range(len(vocab)):
    for j in range(i+1, len(vocab)):
        jaccard = jaccard_matrix[i, j]
        if jaccard > 0:
            nodes.append([re_vocab[i], re_vocab[j], word_cnt[re_vocab[i]], word_cnt[re_vocab[j]], jaccard])
            
len(nodes)

In [ ]:
G = nx.Graph()
G.nodes(data=True)

for pair in nodes:
    node_x, node_y, node_x_cnt, node_y_cnt, jaccard = pair[0], pair[1], pair[2], pair[3], pair[4]
    if not G.has_node(node_x):
        G.add_node(node_x, count=node_x_cnt)
    if not G.has_node(node_y):
        G.add_node(node_y, count=node_y_cnt)
    if not G.has_edge(node_x, node_y):
        G.add_edge(node_x, node_y, weight=jaccard)
        
plt.figure(figsize=(15,15))
pos = nx.spring_layout(G, k=0.1)

node_size = [d['count']*100 for (n,d) in G.nodes(data=True)]
nx.draw_networkx_nodes(G, pos, node_color='cyan', alpha=1.0, node_size=node_size)
nx.draw_networkx_labels(G, pos, fontsize=14, font_family='Droid Sans Japanese')

edge_width = [d['weight']*10 for (u,v,d) in G.edges(data=True)]
nx.draw_networkx_edges(G, pos, alpha=0.2, edge_color='black', width=edge_width)

plt.axis('off')
plt.show()